In [1]:
%load_ext autoreload
%autoreload 2"

In [2]:
import os

# File paths for current state (x), rewards (y), and next states (z)
basefoldername = 'C:/Users/ICSL/Desktop/elen6885/'
foldername_x = basefoldername + 'x_folder/'
foldername_y = basefoldername +  'y_folder/'
foldername_z = basefoldername + 'z_folder/'
foldername_a = basefoldername + 'a_folder/'
#foldername_x = basefoldername + 'testfolder_x/'
#foldername_y = basefoldername +  'testfolder_y/'
#foldername_z = basefoldername + 'testfolder_z/'
#foldername_a = basefoldername + 'testfolder_a/'

# Aggregate current state (x), rewards (y), next states (z) files, and action (a)
aggregate_x_filepath = 'C:/Users/ICSL/Desktop/elen6885/x.csv'
aggregate_y_filepath = 'C:/Users/ICSL/Desktop/elen6885/y.csv'
aggregate_z_filepath = 'C:/Users/ICSL/Desktop/elen6885/z.csv'
aggregate_a_filepath = 'C:/Users/ICSL/Desktop/elen6885/a.csv'
aggregate_x_raw_filepath = 'C:/Users/ICSL/Desktop/elen6885/x_raw.csv'
aggregate_z_raw_filepath = 'C:/Users/ICSL/Desktop/elen6885/z_raw.csv'

# Weight for detecting person (alpha) and for not having camera on (beta)
alpha = [0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3]
beta = [1]

# Environment properties
num_cameras = 10
num_people = 6
num_features = (num_cameras * 4) + (num_people * 2)
num_actions = (7**3)**10
num_states = num_cameras * 4

In [3]:
import re

# Read in all files from current state, rewards, and next states
files_x = os.listdir(foldername_x)
files_y = os.listdir(foldername_y)
files_z = os.listdir(foldername_z)
files_a = os.listdir(foldername_a)

file_aggregate_x_handler = open(aggregate_x_filepath, 'w+')
file_aggregate_z_handler = open(aggregate_z_filepath, 'w+')
file_aggregate_a_handler = open(aggregate_a_filepath, 'w+')
file_aggregate_x_raw_handler = open(aggregate_x_raw_filepath, 'w+')
file_aggregate_z_raw_handler = open(aggregate_z_raw_filepath, 'w+')

# Aggregate all data into one file
for i in range(len(files_z)):
    
    # Open next state files and write to aggregate
    current_z_filepath = foldername_z + files_z[i]
    file_z_handler = open(current_z_filepath, 'r')
    file_z_lines = file_z_handler.readlines()
    file_aggregate_z_raw_handler.writelines(file_z_lines)
    
    # Check and see if there are any numbers very large in magnitude. If so, set it to -1
    for line in file_z_lines:
        state_str = line.strip().split(',')
        state = [float(x) for x in state_str]
    
        for k in range(len(state)):
            if(state[k] < -1000):
                state[k] = -1
        
        # Only use camera states
        state = state[0: num_states]
        
        # Normalize states to be between -1 and 1
        for j in range(num_cameras):
            index = j * 4
            state[index] = state[index] / 30 # Pan is between -30 and 30
            
            index = index + 1
            state[index] = state[index] / 30 # Tilt is between -30 and 30
            
            index = index + 1
            state[index] = state[index] / 100 # Zoom is between -100 and 100
        
        # Write line to file
        new_line = ','.join([str(x) for x in state])
        file_aggregate_z_handler.write(new_line + '\n')
    
    # Find the corresponding current state file
    files_name_re = re.search('_(.*)', files_z[i])
    files_x_name = 'x_' + files_name_re.group(1)
    current_x_filepath = foldername_x + files_x_name
    file_x_handler = open(current_x_filepath, 'r')
    file_x_lines = file_x_handler.readlines()
    file_aggregate_x_raw_handler.writelines(file_x_lines)
    
    # Check and see if there are any numbers very large in magnitude. If so, set it to -1
    for line in file_x_lines:
        state_str = line.strip().split(',')
        state = [float(x) for x in state_str]
    
        for k in range(len(state)):
            if(state[k] < -1000):
                state[k] = -1
        
        # Only use camera states
        state = state[0: num_states]
        
        # Normalize states to be between -1 and 1
        for j in range(num_cameras):
            index = j * 4
            state[index] = state[index] / 30 # Pan is between -30 and 30
            
            index = index + 1
            state[index] = state[index] / 30 # Tilt is between -30 and 30
            
            index = index + 1
            state[index] = state[index] / 100 # Zoom is between -100 and 100
            
        
        # Write line to file
        new_line = ','.join([str(x) for x in state])
        file_aggregate_x_handler.write(new_line + '\n')
    
    # Find the corresponding action file
    files_a_name = 'a_' + files_name_re.group(1)
    current_a_filepath = foldername_a + files_a_name
    file_a_handler = open(current_a_filepath, 'r')
    file_a_lines = file_a_handler.readlines()
    file_aggregate_a_handler.writelines(file_a_lines)
    
    # If we are at the last file, don't write a new line
    #if(i != len(files_z) - 1):
        #file_aggregate_a_handler.write('\n')
        #file_aggregate_x_raw_handler.write('\n')
        #file_aggregate_z_raw_handler.write('\n')
    
    file_x_handler.close()
    file_z_handler.close()
    file_a_handler.close()

file_aggregate_x_handler.close()
file_aggregate_z_handler.close()
file_aggregate_a_handler.close()
file_aggregate_x_raw_handler.close()
file_aggregate_z_raw_handler.close()
    
# Compute rewards by looping through all alpha and beta values to generate rewards for each alpha, beta pair
for a, b in [(a, b) for a in alpha for b in beta]:
    
    # Open aggregate rewards file for current alpha, beta pair
    current_y_filename = 'y_a_' + str(a) + '_b_' + str(b) + '.csv'
    current_y_filepath = foldername_y + current_y_filename
    file_y_handler = open(current_y_filepath, 'w+')
    
    with open(aggregate_x_raw_filepath, 'r') as x, open(aggregate_z_raw_filepath, 'r') as z:
        
        while True:
            x_line = x.readline()
            z_line = z.readline()
            
            if (not x_line) or (not z_line) or x_line[0] == '' or z_line[0] == '':
                break
        
            try:
                x_state_str = x_line.strip().split(',')
                x_state = [float(x) for x in x_state_str]
            except ValueError:
                break
        
            # Read in next state
            try:
                z_state_str = z_line.strip().split(',')
                z_state = [float(x) for x in z_state_str]
            except ValueError:
                break
            
            # Generate state rewards: alpha * number of people detected - beta * number of cameras that captured a frame
            # State action rewards: next state reward - curr state reward
            # Rewards vector:
            #    camera 1:
            #        pan
            #        tilt
            #        zoom
            #        on/off
            #    camera 2:
            #        .
            #        .
            #        .
            #    camera num_cameras:
            #        pan
            #        tilt
            #        zoom
            #        on/off
            #    person 1:
            #        x_coord
            #        y_coord
            #    person 2:
            #        .
            #        .
            #        .
            #    person num_people:
            #        pan
            #        tilt
            #        zoom
            #        on/off
            # Generate rewards for number of people detected
            x_reward = 0
            z_reward = 0
            for i in range(num_people):
                x_index = (4 * num_cameras) + (i * 2) + 0
                y_index = x_index + 1
                
                # If both y and x indices are extremely negative, then the person was not detected
                if(not (x_state[x_index] <= -1000 and x_state[y_index] <= -1000)):
                    x_reward = x_reward + (a * 1)
                if(not (z_state[x_index] <= -1000 and z_state[y_index] <= -1000)):
                    z_reward = z_reward + (a * 1)
                
            # Generate rewards for number of cameras that captured a frame
            for i in range(num_cameras):
                index = (i * 4) + 3
                
                # Looking at the on/off flag (which will be 1 if frame was captured, 0 if off)
                if(x_state[index]):
                    x_reward = x_reward - (b * 1)
                else:
                    x_reward = x_reward #+ (b * 1)
                if(z_state[index]):
                    z_reward = z_reward - (b * 1)
                else:
                    z_reward = z_reward #+ (b * 1)
            
            # Write to file
            reward = z_reward - x_reward
            file_y_handler.write(str(reward) + '\n')
        
    file_y_handler.close()

In [4]:
x = [1, 3, 4, 5]
print(x[0:len(x)])

[1, 3, 4, 5]


In [5]:
print(files_z)

['z_0.csv', 'z_1.csv']


In [6]:
print(str(x_line))

In [7]:
file_y_handler.close()